# TODO: (18 june)
Check Roadmap!


In [1]:
# log_with_config.py
import logging
import logging.config
 
"""
Based on http://docs.python.org/howto/logging.html#configuring-logging
"""
logging.config.fileConfig('logs/logging.conf')
logger = logging.getLogger("fintech")

logger.info("WELCOME TO FINTECH! - Program started")

2018-03-08 23:35:52,878 - fintech - INFO - Program started
2018-03-08 23:35:52,892 - fintech - INFO - Done!


In [1]:
from datetime import datetime

import numpy as np
import pandas as pd
from PyPDF2 import PdfFileReader
import re
import tabula

ImportError: No module named PyPDF2

In [ ]:
# Defining functions

def convert_amount(df, variable, errors = "raise"):   
    df[variable + "_tmp"] = df[variable].str.replace("*", "")
    df[variable + "_tmp"] = df[variable + "_tmp"].str.replace(".", "")
    df[variable + "_tmp"] = df[variable + "_tmp"].str.replace(",", ".")
    df[variable + "_tmp_int"] = pd.to_numeric(df[variable + "_tmp"], errors = errors)
    return df

def convert_PDFunit(measure, unit="inches"):
    if unit=="inches":
        return measure*72
    elif unit=="cm":
        return measure*28.3
    else:
        return None

# Regular Account

In [ ]:
regular_columns = ["date", "value", "operation_details", "debit", "credit"]

def count_pages(file_path):
    f = open(file_path, mode='rb' )
    try:
        reader = PdfFileReader(f)
        n_pages = reader.getNumPages() 
    finally:
        f.close()
    return n_pages

def parse_pdf(file_path, page="all", area=None, guess=True, columns=None):
    # Read pdf into DataFrame          
    if pdf_pages!="all":
        print("\t\t[INFO] Parsing page " + str(page))
        
    if area is not None:
        guess = False
    else:
        print("\t\t\t[WARNING] Parsing guessing table zone, check results.")
    
    df = tabula.read_pdf(file_path, 
                         output_format='dataframe',
                         encoding = "ISO-8859-1",
                         java_options=None, 
                         pandas_options = {'header' : None},
                         multiple_tables=False, 
                         nospreadsheet = True,
                         guess = guess,
                         pages = page,
                         area = area,
                         columns = columns
                        )
    if df is not None:
        df = clean_df(df, file_path)
    else:
        print("\t\t\t[WARNING] No data was parsed from page ["+str(page)+"] of file ["+file_path+"]")
    return df

def clean_df(df, file_path):
    try:
        df.columns = regular_columns
        # Extracting attributes: from files
        match = re.search(r'_(\d{4}\d{2}\d{2})\.', file_path)
        df["bill_date"] = datetime.strptime(match.group(1), '%Y%m%d').date()
        df["account_ref"] = re.search(r'releve_(\d+)_', file_path).group(1)
    except Exception:
        print("\t\t[WARNING] A problem was found cleaning page ["+str(page)+"] of file ["+file_path+"]")
        pass
    return df

In [ ]:
data_path = 'data/pdf/'
regular_pattern = r'_00050814111_'
area_first = [386.412,28.609,734.926,568.843]
area_first = [355.202,29.352,731.21,567.357] #with header

area_middle = [166.454,28.609,619.745,567.357]
columns_bounds = [77.654,130.414,413.535, 491.561]

l = os.listdir(data_path)
r = re.compile(regular_pattern)
regular_files = list(filter(r.search, os.listdir(data_path)))

dfs = []
dfs_prob = []
print("There are " + str(len(regular_files)) + " files in the folder.")
print("Out of these, " + str(len(files)) + " are being parsed")

files=regular_files
for i, file in enumerate(files): 
    file_path = os.path.join(data_path, file)
    print("[INFO] Parsing file "+file_path+". ["+str(i)+"/"+str(len(files))+"]...")

    pdf_pages = count_pages(file_path)
    
    if pdf_pages!="all":
        print("\t[INFO] Individual page parsing. Total of ",str(pdf_pages)," pages to parse")
        
    for page in [i+1 for i in range(pdf_pages)]:
        try:
            if page == 1:
                # first page, always same area
                print("\t\t\t[INFO] First page settings...")
                df_temp = parse_pdf(file_path, page, area = area_first, columns = columns_bounds)
                dfs.append([df_temp])
            elif page == pdf_pages:
                # last page, guessing!
                print("\t\t\t[INFO] Last page settings...")
                df_temp = parse_pdf(file_path, page, columns = columns_bounds)
                dfs[i].append(df_temp)
            else :
                # middle pages
                print("\t\t\t[INFO] Middle page settings...")
                df_temp = parse_pdf(file_path, page, area = area_middle, columns = columns_bounds)
                dfs[i].append(df_temp)
            
        except Exception as e:
            dfs_prob.append(df_temp)
            print("\t[WARNING] File:"+file_path+" skipped due to error: '"+str(e)+"'. Continuing...")
            print("\t\t[ERROR] "+str(e)+" Continuing...")

#df = pd.concat(dfs).reset_index()
print("Done!")

In [ ]:
dfs[2][1]

In [ ]:
dfs_ok = []
dfs_df = []
for file in dfs:
    dfs_ok.append(pd.concat([x for x in file if x is not None]))
    
    
for file in dfs_ok:
    print(file.shape)

In [ ]:
df = pd.concat(dfs_ok)
print(df.shape)
# Getting rid of the parsed headers
df = df[df.date!="Date"]

In [ ]:
# converting amounts into integer values
convert_amount(df, "credit")
convert_amount(df, "debit")

convert_amount(df, "table_solde", errors = "coerce")

df

In [ ]:
# concatenating text from extra-column to operations column
df["table_solde_tmp_str"] = df["table_solde_tmp"][df["table_solde_tmp_int"].isnull()]
df["table_solde_tmp_str"] = df["table_solde_tmp_str"].fillna(value = "")
df["operation_details"] = df["operation_details"].fillna(value = "")

df["operation"] = df["operation_details"] + df["table_solde_tmp_str"]

df.head()

In [ ]:
# Creating definitive debit column: Attributing amounts from extra-column to debit column
df[df["table_solde_tmp_int"].notnull()] # TODO: raise error here when debit_final is not null, and there is also a value at debit_tmp_int

df["debit_final"] = np.where(df["table_solde_tmp_int"].notnull(), 
                              df["table_solde_tmp_int"],
                              df["debit_tmp_int"]
                             )
df

In [ ]:
# Creating definitive credit column: No actions at the time
df["credit_final"] = df["credit_tmp_int"]
df

In [ ]:
# Ignoring last row of total movements
df[df["operation"]=="TOTAUX DES MOUVEMENTS"]
df[~df["credit"].isnull() & ~df["debit"].isnull()]

total_values = ~df["credit"].isnull() & ~df["debit"].isnull()
total_tag = df["operation"]=="TOTAUX DES MOUVEMENTS"

df = df[~(total_values | total_tag)]
df

In [ ]:
# Treating date fields and converting to datetime format
df["date_op"] = df.loc[:,"date"].str.split(" ").str.get(0)

df["value_op"] = df["date"].str.split(" ").str.get(1)
# TODO: better manage the errors here. There are weird-non-date values
df["date_op_dt"] = pd.to_datetime(df["date_op"], dayfirst=True, errors="coerce")
df["value_op_dt"] = pd.to_datetime(df["value_op"], dayfirst=True, errors="coerce")
df

In [ ]:
# Cleaning columns
df.drop(["operation_details", 
         "debit", "debit_tmp", "debit_tmp_int",
         "credit", "credit_tmp", "credit_tmp_int",
         "table_solde", "table_solde_tmp", "table_solde_tmp_str", "table_solde_tmp_int",
         "date", "value", "date_op", "value_op"         
        ], axis=1, inplace = True)

df.rename(columns = {"debit_final" : "debit",
                     "credit_final" : "credit",
                     "date_op_dt" : "date",
                     "value_op_dt" : "value",          
          }, inplace = True)
df

In [ ]:
# Cleaning rows
status_pattern = r'^\*\*\* SOLDE'
df = df[~df.operation.str.contains(status_pattern)]

# Filtering all NaN rows
df.operation = df.operation.replace('', np.nan)
df.dropna(axis='index', how='all')

In [ ]:
dfcopy=df

In [ ]:
# Re-arranging rows and columns
transaction_mask = ~(df.debit.isnull() & df.credit.isnull())
transaction_index = pd.to_numeric(df.loc[transaction_mask,:].index)
details_index = pd.to_numeric(df.loc[~transaction_mask,:].index)

df["transaction_index"]= np.nan
df.loc[transaction_index, "transaction_index"] = transaction_index
df["transaction_index"]=df.transaction_index.fillna(method='ffill')

df["operation_detail"]=np.nan
df.loc[details_index, "operation_detail"] = df.loc[details_index, "operation"]

operation_details=df[~df.operation_detail.isnull()].groupby('transaction_index')['operation_detail'].apply(lambda x: '|'.join(x))
df['op_details']=operation_details

df_compact = df[transaction_mask].reset_index().drop(["index", "transaction_index", "operation_detail"], axis='columns')
df_compact

In [ ]:
op_types = {
    "transfer" : "VIR",
    "check" : "CHEQUE",
    "charge" : "PRELEVEMENT",
    "" : "COTISATION",
    "" : "DEBIT",
    "" : "ARRETE",
    "" : "FRAIS",
    "" : "CARTE",
}

df_compact["op_type"] = df_compact.operation.str.extract(r'(?P<numbers>[0-9 ]*)(?P<type>[A-Z]*)').loc[:,"type"]
df_compact.op_type.unique()

In [ ]:
# Extracting attributes: from operation details
df_compact["from"] = df_compact.op_details.str.extract(r'DE:(?P<de>[^|]+)')
df_compact["id"] = df_compact.op_details.str.extract(r'ID:(?P<de>[^|]+)')
df_compact["to"] = df_compact.op_details.str.extract(r'POUR:(?P<de>[^|]+)')
df_compact["ref"] = df_compact.op_details.str.extract(r'REF:(?P<de>[^|]+)')
df_compact["motive"] = df_compact.op_details.str.extract(r'MOTIF:(?P<de>[^|]+)')
df_compact["at"] = df_compact.op_details.str.extract(r'CHEZ:(?P<de>[^|]+)')
df_compact["mandat"] = df_compact.op_details.str.extract(r'MANDAT:(?P<de>[^|]+)')

In [ ]:
df_compact

In [ ]:
logger.info("Done!")